In [71]:
!pip install TextBlob

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [79]:
#Using Tweets from voters and twitter users
biden = pd.read_csv('/content/drive/MyDrive/Biden.csv')
trump = pd.read_csv('/content/drive/MyDrive/Trump.csv')

In [80]:
biden.head()

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr
3,penblogger,@JoeBiden It's clear you didnt compose this tw...
4,Aquarian0264,@JoeBiden I will vote in person thank you.


In [83]:
trump.head()

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...
4,James09254677,After 4 years you think you would have figure...


In [89]:
trump['text'][100]

' Thank you President Trump for all you are doing to help the American people. First President in hi… https://t.co/MndKc1gtJB'

In [87]:
biden['text'][1000]

'@JoeBiden This is stupid tweet. Don’t tell the people what trump needs to do, tell them what you’ll do. What if he… https://t.co/bv4N7IBCMJ'

In [92]:
from textblob import TextBlob

text_sample = trump['text'][100]

blob = TextBlob(text_sample)

sentiment_polarity = blob.sentiment.polarity
sentiment_subjectivity = blob.sentiment.subjectivity

print("Sentiment Polarity:", sentiment_polarity)
print("Sentiment Subjectivity:", sentiment_subjectivity)

Sentiment Polarity: 0.125
Sentiment Subjectivity: 0.16666666666666666


In [93]:
text_sample = biden['text'][1000]

blob = TextBlob(text_sample)

sentiment_polarity = blob.sentiment.polarity
sentiment_subjectivity = blob.sentiment.subjectivity

print("Sentiment Polarity:", sentiment_polarity)
print("Sentiment Subjectivity:", sentiment_subjectivity)

Sentiment Polarity: -0.7999999999999999
Sentiment Subjectivity: 1.0


In [98]:
def polarity(review):
    return TextBlob(review).sentiment.polarity

In [99]:
biden["Polarity"]=biden["text"].apply(polarity)
trump["Polarity"]=trump["text"].apply(polarity)

In [100]:
trump["sentiment"]=np.where(trump["Polarity"]>0,"Positive","Negative")
trump["sentiment"][trump["Polarity"]==0]="Neutral"
trump.head()

<ipython-input-100-6bbb6af244fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trump["sentiment"][trump["Polarity"]==0]="Neutral"


,user,text,Polarity,sentiment
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05,Positive
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00,Neutral
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50,Negative
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...,0.50,Positive
4,James09254677,After 4 years you think you would have figure...,0.00,Neutral


In [101]:
biden["sentiment"]=np.where(biden["Polarity"]>0,"Positive","Negative")
biden["sentiment"][biden["Polarity"]==0]="Neutral"
biden.tail()

<ipython-input-101-b0788d6bf0f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biden["sentiment"][biden["Polarity"]==0]="Neutral"


,user,text,Polarity,sentiment
2535,meryn1977,@JoeBiden You'll just try to calm those waters...,0.15,Positive
2536,BSNelson114,@JoeBiden 96 days 96 dias #VoteJoeBiden2020 #...,0.00,Neutral
2537,KenCapel,@JoeBiden YOU THINK YOU CAN DO THAT??? YOU CAN...,0.00,Neutral
2538,LeslyeHale,@JoeBiden Trump wants our children back at sch...,0.10,Positive
2539,rerickre,"@JoeBiden ... and I know, because it’s much co...",0.20,Positive


In [102]:
print(trump.shape)
print(biden.shape)

(2788, 4)
(2540, 4)


In [103]:
trump.drop(trump[trump["Polarity"]==0].index, inplace= True)
print(trump.shape)

(1324, 4)


In [104]:
biden.drop(biden[biden["Polarity"]==0].index, inplace= True)
print(biden.shape)

(1031, 4)


In [105]:
np.random.seed(10)
no=293
drop_index=np.random.choice(trump.index, no, replace=False)
trump=trump.drop(drop_index)

print(trump.shape)

(1031, 4)


In [106]:
print(trump.shape)
print(biden.shape)

(1031, 4)
(1031, 4)


In [107]:
count_trump=trump.groupby('sentiment').count()
print(count_trump)

count_biden=biden.groupby('sentiment').count()
print(count_biden)

           user  text  Polarity
sentiment                      
Negative    462   462       462
Positive    569   569       569
           user  text  Polarity
sentiment                      
Negative    407   407       407
Positive    624   624       624


In [110]:
#The Presidents analysis
name=["Trump","Biden"]
list_pos=[count_trump['Polarity'][1],count_biden['Polarity'][1]]
list_neg=[count_trump['Polarity'][0],count_biden['Polarity'][0]]


import plotly.graph_objects as go

fig=go.Figure(data=[
    go.Bar(name='+ve',x=name, y=list_pos),
    go.Bar(name='-ve', x=name, y=list_neg)
])

fig.update_layout(barmode='group')
fig.show()

In [124]:
#Trump had less positive tweets than Biden and had more negative tweets than Biden
#This shows that Biden was more favourite candidate than Trump in the elections.